# Marc tools

## Import

In [ ]:
import os
import plotly
import importlib
import re
import h5py
import parquet

import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go
import marc_tools as mt
import plotly.express as px

from pathlib import Path

# Set up plotly
plotly.offline.init_notebook_mode()

pio.templates['custom'] = go.layout.Template(
    layout_paper_bgcolor='rgba(0,0,0,0)',
    layout_plot_bgcolor='rgba(0,0,0,0)',
    ) 

# Change background color if using VSC
if 'VSCODE_CWD' in os.environ:
    pio.templates.default = 'plotly+custom'

In [ ]:
importlib.reload(mt.scf)
importlib.reload(mt.general)
importlib.reload(mt.bflex)

In [ ]:
# Define folders
project_dir = Path(r'C:\Projects\1993_Pierce_GI_riser')
bflex_dir = project_dir / 'Bflex/Local/Top_300bar'

In [ ]:
# Find curvature
for curv in bflex_dir.glob(r'*elcurvY.mpf'):
    df_curv = mt.bflex.mpf_reader(curv)

df_curv.columns = ['curv']

# Find mpf_files
df_mpf = pd.DataFrame()

for mpf in bflex_dir.glob(r'*nosl?.mpf'):
    
    df_temp = mt.bflex.mpf_reader(mpf)
    df_mpf = pd.concat([df_mpf, df_temp], axis=1)

# Manipulate df
df = pd.DataFrame()

for i in range(17, 33):
    df[f'P{i}'] = df_mpf.filter(regex=f'{i}').max(axis=1)

df_stress = pd.concat([df_curv, df], axis=1)

# Interpolate curvature
df = pd.DataFrame({'curv' : [i/100 for i in range(0, 21)]})

for col in df_stress.columns[1:]:
    df[col] = np.interp(df.curv, df_stress.curv, df_stress[col])

# df_new.columns = ['Curvature'] + [i*22.5 for i in range(0, 16)]
df

In [ ]:
# Plot
df_plot = df[df.columns[1:]].set_index(df.curv)
fig = go.Figure(data=[go.Surface(z=df_plot)])
fig.update_layout(title='Stress', autosize=False,
                  width=1000, height=800)
fig.show()

In [ ]:
# Bad incs
bad_incs = [26, 28, 29, 38, 39, 40, 46, 47, 51, 57, 64, 65, 67]

# Read Marc files
marc_dir = Path(r'\\nas-ask-001\Projects\1993_Pierce_GIR_fatigue\Marc\jupyter_notebook')
marc_files = marc_dir.glob('*stresses_inner_ta_inc*[0-9].csv*')

# Read list with coordinates per node
df_list = pd.read_csv(marc_dir / 'angles.csv').iloc[:,1:]
df_list.drop('node_index', axis=1, inplace=True)

# Iterate over files and create dataframe
df = pd.DataFrame()

for mf in marc_files:
    # Read increment
    inc = int(re.findall(r'\d+', mf.stem)[0])

    if inc in bad_incs:
        continue
    
    # Read dataframe and add x, y and z coordinate
    df_read = pd.read_csv(mf, index_col=None).iloc[:,1:]

    if all(df_read['node'] == df_list['node_id']):
        df_read = pd.concat([df_read, df_list[['z', 'theta']]], axis=1)

    df = pd.concat([df, df_read], axis=0)

df.sort_values(['inc', 'z', 'theta'], inplace=True)


In [ ]:
dfn = df.where(df['inc'] < 10).dropna()

dfn['diff'] = dfn.groupby('inc')['theta'].diff().gt(3).cumsum()

dfg = dfn.groupby(['inc', 'diff']).max()
dfg.reset_index(inplace=True)
dfg.drop(['diff'], axis=1, inplace=True)
dfg

In [ ]:
df_res = pd.DataFrame()

for inc in range(0, 70, 10):
    
    if inc < 10:
        inc_start = inc
        inc_end = inc + 10
    else:
        inc_start = inc + 1
        inc_end = inc + 10

    # print(f'Checking increments {inc_start} <= inc <= {inc_end}')

    dfn = df.where(df['inc'] <= inc_end).where(df['inc'] >= inc_start).dropna()

    dfn['diff'] = dfn.groupby(['inc', 'z'])['theta'].diff().gt(3).cumsum()

    dfg = dfn.groupby(['inc', 'diff']).max()
    dfg.reset_index(inplace=True)
    dfg.drop(['diff'], axis=1, inplace=True)

    df_res = pd.concat([df_res, dfg], axis=0)


In [ ]:
df_res

In [ ]:
# Plotly express
fig = px.line_polar(df_res, r='stress', theta='theta', line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
fig = go.Figure()

for inc in range(0, 10, 10):
    fig.add_trace(go.Scatterpolar(
        r=df_res['stress'].where(df_res['inc']==inc),
        theta = df_res['theta'].where(df_res['inc']==inc),
        fill='toself',
        name=f'Inc {inc}'
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[200, 500]
    )),
  showlegend=False
)
fig.update_layout(title='Stress', autosize=False,
                  width=1000, height=800)
fig.show()

## Read marc input

In [ ]:
mf = Path(r'\\nas-ask-001\Projects\1993_Pierce_GIR_fatigue\Marc\analysis\NOV_8inch_GIR_3p_with_bs_g3_job1.h5')

In [ ]:
with h5py.File(mf, 'r') as f:
    print(f'Keys {f["Marc"]["Results"]["Element"]["Tensor"]["Cauchy Stress in Preferred Sys"].keys()}')

